# Filtrado de hilos de 4chan

En esta sección se filtran los hilos de 4chan que contienen palabras o frases que se han encontrado en la literatura como indicadores de tendencias suicidas.

La literatura consultada se puede consultar en la sección de [Referencias](./README.md#referencias) del archivo `README.md`.

In [ ]:
# Cargamos las palabras que queremos filtrar
